In [2]:
import pandas as pd
import urllib
import sqlalchemy as sa
from datetime import datetime
import numpy as np

# 1. Multiparity, lambing ease, mikiness phenotypes: general infromation

There are three ovine selection models which we are currently intereseted in: 

1. Multiparity. The things to be assessed per lambing: total number of lambs born, the number of lambs born alive, the number of male lambs.
2. Lambing ease, in points: 1 - easy lambing, 2 - complicated lambing.
3. Milkiness: weight of all lambs born in one lambing at the age of breaking in to milk minus weight of ones at the moment of birth.

To start calculation of these selection models we need both genotype and phenotype data. This page provides a way to obtain phenotypes only.

Phenotypic data is taken from out SQL DB.

# 2. Connection to the DataBase, SQL queries and the result dataframe

In [3]:
params = urllib.parse.quote_plus( 
    'Driver={ODBC Driver 17 for SQL Server};' 
    'Server=bestserver;' 
    'Database=ovine;' 
    'UID=MyUID;' 
    'PWD=123;')

In [ ]:
engine = sa.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

The lambing data is taken with the SQL query below:

In [ ]:
lambs = """SELECT c.animal_no, a.birth_date, p.probe_code, a.breed as mother_breed, c.ease, c.farm_code, c.operation_date, YEAR(c.operation_date) AS lambing_year, 
    CASE 
        WHEN MONTH(c.operation_date) BETWEEN 3 AND 5 THEN 'SPRING'
        WHEN MONTH(c.operation_date) BETWEEN 6 AND 8 THEN 'SUMMER'
        WHEN MONTH(c.operation_date) BETWEEN 9 AND 11 THEN 'FALL'
        WHEN MONTH(c.operation_date) IN(1,2,12) THEN 'WINTER'
    END lambing_season,
    c.breed as lamb_breed, c.new_animal_no1, c.sex1, c.weight1, c.stillborn1, c.new_animal_no1, c.sex2, c.weight2, c.stillborn2,
    c.new_animal_no3, c.sex3, c.weight3, c.stillborn3, c.new_animal_no4, c.sex4, c.weight4, c.stillborn4
FROM LAMBINGS AS c
INNER JOIN ANIMALS AS a ON c.animal_no = a.animal_no
INNER JOIN PROBE_CARD AS p ON c.animal_no = p.animal_no
WHERE c.abort=0 AND c.genetic_anomaly1=0 AND c.genetic_anomaly2=0 AND c.genetic_anomaly3=0 
AND c.genetic_anomaly4=0 AND c.IVF=0 AND a.breed IN ('DORPER', 'ROMANOV')

"""

The dataframe obtained have the following structure:

| Column name             | Description                             |
|-------------------------|-----------------------------------------|
| animal_no               | Mother number                           |
| birth_date              | Mother's birth date                     |
| probe_code              | Code of the probe taken from the mother |
| mother_breed            | Mother breed                            |
| farm_code               | Code of the farm where lambing occurred |
| operation_date          | Lambing date                            |
| lambing_year            | Lambing year                            |
| lambing_season          | Lambing season                          |
| lamb_breed              | Lamb breed                              |
| new_animal_no<number>   | Lamb number                             |
| sex<number>             | Lamb sex                                |
| weight<number>          | Lamb weight                             |
| stillborn<number>       | Lamb stillbirth mark                    |
    
The information about lambs is organized in the following way: maximal number of lambs born and registered is 4. For each lamb there are 4 columns describing its number, sex, weight and stillbirth. If some columns are left unfilled, it means that these lambs were not born. 

Basic function to obtain a DF from the SQL query:

In [ ]:
def sql_reader(sql):
    with engine.connect() as con:
        return pd.read_sql(sql, con)
    return None

# 3. Selection of valid lambing records

For further work we need to select valid records for each lambing.

The data is considered valid and taken without any additional checks if there are 130 or more days between subsequent lambings.

If the subsequent lambings occurred within 129 days, in this case the whole group of such records needs validation. In such cases we consider that there was a fact of lambing, but some errors, of technical or human origin, occurred while registration. The record having the biggest number of columns filled is taken from the whole group. If there are 2 or more records equally filled, any of them is taken.

Here are the helper functions to select valid records:

In [ ]:
def _compare_dates(ts_list, mode):
    """
    Helper function to obtain a list of nirmal (valid) or abnormal lambing dates depending of the mode parameter.
    :param ts_list: a sorted list of lambing dates (operation_date) per one ovine mother.
    :param mode: one of two values, 'normal' - to return a list of normal dates, 'abnormal' - to return a list of abnormal dates.
    :return: a list of nomal or abnormal lambing dates.
    """
    ts_list = [str(i)[:10] for i in ts_list]
    ts_list = [datetime.strptime(i, '%Y-%m-%d').date() for i in ts_list]
    
    normal_dates = []
    abnormal_dates = []
    
    if len(ts_list) == 1:
        normal_dates.append(ts_list[0])
    
    else:
        for i in range(len(ts_list)):
            if i == 0:
                if abs(ts_list[i] - ts_list[i+1]).days >= 130:
                    normal_dates.append(ts_list[i])
                else:
                    abnormal_dates.append(ts_list[i])
            elif i == len(ts_list) - 1:
                if abs(ts_list[i] - ts_list[i-1]).days >= 130:
                    normal_dates.append(ts_list[i])
                else:
                    abnormal_dates.append(ts_list[i])
            else:
                if abs(ts_list[i] - ts_list[i+1]).days >= 130 and abs(ts_list[i] - ts_list[i-1]).days >= 130:
                    normal_dates.append(ts_list[i])
                else:
                    abnormal_dates.append(ts_list[i])

    if mode == 'normal':
        return normal_dates
    elif mode == 'abnormal':
        return abnormal_dates
    
    return "A proper mode was not chosen"

In [ ]:
def _group_abnormal_dates(abnormal_dates):
    """
    Helper function to obtain groups of abnormal dates. 
    :param abnormal_dates: a sorted list of abnormal dates.
    :return: a list of abnormal date groups.
    """
    abnormal_dates = [str(i)[:10] for i in abnormal_dates]
    abnormal_dates = [datetime.strptime(i, '%Y-%m-%d').date() for i in abnormal_dates]
    
    groups = []
    group = 1
    
    for i in range(len(abnormal_dates)):
        if i != len(abnormal_dates) - 1 and abs(abnormal_dates[i] - abnormal_dates[i+1]).days >= 130:
            groups.append(group)
            group += 1
        else:
            groups.append(group)
    return groups

In [ ]:
def _get_lambing_data(labms_df, mode='all'):
    """
    Helper function to select lambing records including initially valid records and any of the most filled record for a groupd having abnormal dates.
    :param lambs_df: a lambing dataframe taken from the SQL query.
    :param mode: one of two values: - "first" - if the model is applied to the first lambings only.
                                    - "all" - if the model is applied to all lambings.
    :return: a dataframe for firs or all lambings depending on the mode parameter.
    """
    # Count notnull values for the lamb breed, weight and stilbirths (data_count).
    labms_df['data_count'] = labms_df[['calve_breed', 'weight1', 'stillborn1', 'weight2', 'stillborn2', 
                                    'weight3', 'stillborn3', 'weight4', 'stillborn4']].notnull().sum(axis=1)
    # Choose the fullest record for each date, no matter valid or not.
    labms_df = labms_df.sort_values(['animal_no', 'operation_date', 'data_count'], 
                ascending=False).groupby(['animal_no', 'operation_date'], as_index=False).first()
    # Obtaining a DF with the list of normal and abnormal dates. The latter are grouped.
    # If the abnormal dates belong to diferent lambings, so the groups will be different, and one record for each group is taken afterwards.
    animal_dates = labms_df[['animal_no', 'operation_date']].groupby(['animal_no'], as_index=False).agg(lambda x: list(x))
    animal_dates['normal_dates'] = animal_dates['operation_date'].apply(_compare_dates, mode='normal')
    animal_dates['abnormal_dates'] = animal_dates['operation_date'].apply(_compare_dates, mode='abnormal')
    animal_dates['abnormal_groups'] = animal_dates['abnormal_dates'].apply(_group_abnormal_dates)
    # Obtaining a DF with normal dates only
    normal_dates = animal_dates.loc[animal_dates['normal_dates'].apply(lambda x: len(x)) > 0]
    normal_dates = normal_dates[['animal_no', 'normal_dates']].explode(['normal_dates'])
    normal_dates['normal_dates'] = pd.to_datetime(normal_dates['normal_dates'])
    normal_lambings = normal_dates.merge(calve_entry, how='left', left_on=['animal_no', 'normal_dates'], 
                    right_on=['animal_no', 'operation_date']).drop(['normal_dates'], axis=1)
    # Obtaining a DF with abnormal dates only with the fullest record selected per each group.
    abnormal_dates = animal_dates.loc[animal_dates['abnormal_dates'].apply(lambda x: len(x)) > 0]
    abnormal_dates = abnormal_dates[['animal_no', 'abnormal_dates', 'abnormal_groups']].explode(['abnormal_dates', 'abnormal_groups'])
    abnormal_dates['abnormal_dates'] = pd.to_datetime(abnormal_dates['abnormal_dates'])
    abnormal_lambings = abnormal_dates.merge(calve_entry, how='left', left_on=['animal_no', 'abnormal_dates'], 
                        right_on=['animal_no', 'operation_date']).drop(['abnormal_dates'], axis=1)
    abnormal_lambings = abnormal_lambings.sort_values(['animal_no', 'abnormal_groups', 'data_count'], 
                        ascending=False).groupby(['animal_no', 'abnormal_groups'], as_index=False).first().drop(['abnormal_groups'], axis=1)

    # Concatenating the normal and abnormal dates DFs and sorting by mother number and lambing date.
    all_lambings = pd.concat([normal_lambings, abnormal_lambings], 
                ignore_index=True).sort_values(['Animal No_', 'Operation Date'], ascending=True).drop(['data_count'], axis=1)
    # If the mode='fist', the data for first lambings are chosen only.
    if mode == 'first':
        return all_lambings.groupby(['Animal No_'], as_index=False).first()
    
    return all_lambings

# 4. Multiparity and lambing ease

Multiparity, lambing ease are calculated for each record from the DF received at p. 3.

In [ ]:
def _calc_offsprings(all_lambings, col_num):
    """
    Helper function to calculate the number of born lambs, alive lambs and male lambs per lambing.
    :params all_lambings: DF with lambing data, obtained after _get_lambing_data execution.
    :param col_num: maximal number of lambs to be registered per lambing, in our case it's 4.
    :return: all_lambings with columns containg numbers of born lambs, alive lambs and male lambs
    """
    all_lambings['offspr_total'] = 0
    all_lambings['offspr_alive'] = 0
    all_lambings['offspr_male'] = 0
    
    for i in range(1, col_num+1):
        sex_str = f"sex{i}"
        weight_str = f"weight{i}"
        stb_str = f"stillborn{i}"
        all_lambings.loc[all_lambings[weight_str] != 0, 'offspr_total'] += 1
        all_lambings.loc[(all_lambings[weight_str] != 0) & (all_lambings[stb_str] == 0), 'offspr_alive'] += 1
        all_lambings.loc[(all_lambings[weight_str] != 0) & (all_lambings[sex_str] == 0), 'offspr_male'] += 1
    
    return all_lambings

**The main function to obtain all required values for lambing ease and multiparity**

In [ ]:
def mult_ease(sql, col_num=4, lambing_data_mode='all'):
    """
    Contains the logic to calculate data for miltiparity and ease phenotypes.
    :param sql: SQL query to download lambing data from the database
    :param col_num: maximal number of lambs to be registered per lambing, in our case it's 4.
    :param lambing_data_mode: one of two values: - "first" - if the model is applied to the first lambings only.
                                                 - "all" - if the model is applied to all lambings.
    :return: a DF with phenotypes for miltiparity and ease selection models.
    """
    lambs = sql_reader(sql)
    lambs = _get_lambing_data(lambs, lambing_data_mode)
    lambs = _calc_offsprings(lambs, col_num)
    return lambs

In [ ]:
okot_df = mult_ease(calves, calves2, 4, 'all')

# 5. Milkiness

Milkiness is weight of all lambs at the age of breaking in to milk minus weight of all lambs at the moment of birth.

The lambs are taken from their mothers at the age of 90 days. 

In practice it can vary, or there can be several weighings around this age. Therefore we take from the database records with weighings made in the period from 80 to 100 days, and if there are several records withing this time interval, the closest to 90 days is taken.

In [ ]:
sql_weight = """SELECT w.animal_no, w.weight, DATEDIFF(DAY, a.birth_date, w.opertaion_date) AS date_diff
    FROM WEIGHT AS w
    LEFT JOIN ANIMALS AS a ON w.animal_no = a.animal_no
    WHERE DATEDIFF(DAY, a.birth_date, w.operation_date) BETWEEN 80 and 100"""

In [ ]:
def _get_weight_df(sql):
    """
    Helper function to obtain data about weighings closest to 90 days.
    :params sql: SQL query to obtain weight data.
    :return: DF with weight and age of weighing (date_diff).
    """
    weight_df = sql_reader(sql_weight)
    weight_df['date_90'] = abs(weight_df['date_diff'] - 90)
    weight_df = weight_df.sort_values(['Animal No_', 'date_90'], ascending=True).groupby(['Animal No_'], 
                    as_index=False).first().drop(['date_90'], axis=1)
    return weight_df

In [ ]:
def _merge_weaning_weight(okot_df, weight_df, col_num):
    """
    Helper function to merge weight at about 90 days with the lambing DF having weight at birth for each lamb.
    :param okot_df: DF obtained after mult_ease execution.
    :param weight_df: DF after _get_weight_df execution, which contains weight at about 90 days.
    :param col_num: maximal number of lambs to be registered per lambing, in our case it's 4.
    :return: okot_df with merged data of 90 days weight.
    """
    weight_df = weight_df.rename(columns={'animal_no': 'animal_number', 'weight': 'weight_weaning'})
    
    for i in range(1, col_num+1):
        diff_str = f"date_diff{i}"
        weight_str = f"weight_weaning{i}"
        left_merge_str = f"new_animal_no{i}"
        okot_df = okot_df.merge(weight_df, how='left', left_on=left_merge_str, right_on='animal_number')
        okot_df = okot_df.drop(['animal_number'], axis=1)
        okot_df = okot_df.rename(columns={'weight_weaning': weight_str, 'date_diff': diff_str})
        okot_df.loc[okot_df[weight_str].isnull(), weight_str] = 0
        
    return okot_df

**The main function to calculate milkiness**

In [ ]:
def calc_milkiness(okot_df, sql, col_num):
    """
    Calculates milkiness.
    :param okot_df: DF obtained after mult_ease execution.
    :params sql: SQL query to obtain weight data.
    :param col_num: maximal number of lambs to be registered per lambing, in our case it's 4.
    :return: okot_df with calculated milkiness.
    """
        
    weight_df = _get_weight_df(sql)
    okot_df = _merge_weaning_weight(okot_df, weight_df, col_num)
       
    okot_df['nest_weight_birth'] = okot_df['weight1'] + okot_df['weight2'] + okot_df['weight3'] + okot_df['weight4']
    okot_df['nest_weight_weaning'] = okot_df['weight_weaning1'] + okot_df['weight_weaning2'] + okot_df['weight_weaning3'] + okot_df['weight_weaning4']
    
    okot_df['milkiness'] = okot_df['nest_weight_weaning'] - okot_df['nest_weight_birth']
    
    return okot_df

In [ ]:
milky = calc_milkiness(okot_df, sql_weight, 4)